Suppose the spam generator becomes more intelligent and begins producing prose which looks "more legitimate" than before. 

There are numerous ways the prose could become more like legitimate text. For the purpose of this notebook we will simply force the spam data to *drift* by adding the first few lines of Pride and Prejudice to the start of the spam documents in our testing set. We will then see how the trained model responds.  

In [2]:
import pandas as pd
import os.path

df = pd.read_parquet(os.path.join("data", "training.parquet"))

We split the data into training and testing sets, as in the modelling notebooks. We use the `random_state` parameter to ensure that the data is split in the same way as it was when we fit the model. 

In [3]:
from sklearn import model_selection

df_train, df_test = model_selection.train_test_split(df, random_state=43)
df_test_spam = df_test[df_test.label == 'spam'].copy() #filter the spam documents

In [4]:
def add_text(doc, adds):
    """
    takes in a string _doc_ and
    appends text _adds_ to the start
    """
    
    return adds + doc

In [5]:
pride_pred = '''It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.'''

In [6]:
# appending text to the start of the spam
df_test_spam["text"] = df_test_spam.text.apply(add_text, adds=pride_pred)

In [7]:
pd.set_option('display.max_colwidth', -1) #ensures that all the text is visible
df_test_spam.sample(3)

/tmp/ipykernel_16468/1532718575.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) #ensures that all the text is visible


,index,label,text
34706,14706,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.Perfect balance of healthy ingredients. She is a mix between green tea and I never have to be sure she is eating, I just put the bags back in right. You will not be buying again. wasnt thrilled with this coffee. I don't find any aftertaste to them. To date, we've used about 6 tablespoons of this in with their old food back to Purina. I'll not be ordering this stuff! I bought again with the subscribe and save...perfect!! This is a great value considering the price."
31919,11919,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.Also, the instructions on the package anymore, they still use organic corn but use non organic canola is most likely a food allergy involved. Sweet Bay Ray's got it right, then the other substitutes.Seller had item shipped rather quickly. Definitely recommend it. Don't waste your time. I also like that the salba is included in one bag. Then we bought 2- 6 packs!! If I could give this food a bit more potent. All five of us really enjoy swedish fish! So far these are the bomb!! I use the shampoo, blow dry & then use Aveda Light Elements shaping wax."
26746,6746,spam,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?” Mr. Bennet replied that he had not. “But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.” Mr. Bennet made no answer. “Do you not want to know who has taken it?” cried his wife impatiently.It's incredible.I feel so bad that I was shipped Rasberry Chocolate...in a Pink wrapper. OMG....this stuff is yummy in hot cocoa, coffee, latte's...great price and speedy delivery. Get some! I've been buying other brands of Chai looking for the meal I was making, but once I received my shipment last night and my kitty enjoyed every single one.The Chicken Pot Pie variety is a little steep. My kids love them, the chances are high that your cats may or may not have normal reaction to these treats. I was very disappointed. This is the first time i tried them. I am totally satisfied."


We now pass this "drifted" data through the pipeline we created: we compute feature vectors, and we make spam/legitimate classifications using the model we trained. 

In [8]:
from sklearn.pipeline import Pipeline
import pickle, os

## loading in feature vectors pipeline
filename = 'feature_summaries_pipeline.sav'
feat_pipeline = pickle.load(open(filename, 'rb'))

## loading model
filename = 'model_summaries_logistic.sav'
model = pickle.load(open(filename, 'rb'))



In [9]:
pipeline = Pipeline([
    ('features',feat_pipeline),
    ('model',model)
])

## we need to fit the model, using the un-drifted data, as we did in the previous notebooks. 

pipeline.fit(df_train["text"], df_train["label"])



Pipeline(steps=[('features', Pipeline(steps=[('features', SimpleSummaries())])),
                ('model', LogisticRegression(max_iter=4000))])

In [10]:
## we can then go on and make predictions for the drifted spam, using the fitted pipeline above. 
# predict test instances
y_preds = pipeline.predict(df_test_spam["text"])
print(y_preds)

['legitimate' 'legitimate' 'legitimate' ... 'legitimate' 'legitimate'
 'spam']


In [11]:
import numpy as np
np.array(np.unique(y_preds, return_counts = True))

array([['legitimate', 'spam'],
       [3807, 1189]], dtype=object)

The model is worse at classifying drifted data, since this is not what we trained the model on. 

## Exercises
The two models perform very similarly on the "drifted" data in this notebook. Consider alternative types of data drift and see how the models perform: 
1. What happens when fewer words from Pride and Prejudice are appended to the spam? 
2. How about using a completely different excerpt of Austen? 
3. How do the models perform when generic text (neither Austen nor food reviews) is appended to the spam? 